In [2]:
P = Primes()
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m,x):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m,x):
    exn = exNo3c(n+2,m,x)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m,x):
    f = J(n,m,x)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 
def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

def logsAndMultiplicitiesOverFieldTest(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    print gf
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data



def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data

def padListOnRightToLengthK(listo,k):
    length=len(listo)
    padded=listo
   
    if length<k:
        for j in [length..k-1]:
            padded=padded+[0]
           
        return padded
    return listo
            

In [ ]:
data=[]
for m in [3..500]:
    if 10 in divisors(m):
        print(m)
    data=data+[[m,j(120,m)]]
import pickle
wfile = open('/Users/barrybrent/8mar21no6.txt','a')
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()# output snipped 

In [ ]:
import pickle
rfile = open('/Users/barrybrent/8mar21no6.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import time
wfile = open('/Users/barrybrent/10mar21no8.txt','w')
enoughfile=[]
for qpower in [0..120]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=x*s[k][1] 
# times x because, to use polynomialCoefficient, I need a polynomial; not a Laurent series
        cf=polynomialCoefficient(qpower,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[qpower-1,rl]] 
    finish = time.time()
    print([qpower,finish-start])
# -1 because I want to record the coefficients of the qpowers of the original Laurent series
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
# output snipped

In [3]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print len(s)
print s[1]

121
[0, 24*x^3 + 32*x]


In [4]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
residue=0
bound=120
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        zerodata=[]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverFieldTest(poly,prime,sd)
        logs=[]
        digits=[]
        multiplicities=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            multiplicity=lm[j][1]
            digit=log.digits(prime)
            digit= padListOnRightToLengthK(digit,sd)
            logs=logs+[log]
            multiplicities=multiplicities+[multiplicity]
            digits=digits+[digit]
        print
        print (index,sd,td,multiplicities)
        print
        print logs;print;print digits
        print "--------------------------------------------------------------------------------------"

(4) * x * (x + t + 2) * (x + 4*t + 3)

(0, 2, 3, [1, 1, 1])

[-1, 3, 15]

[[-1, 0], [3, 0], [0, 3]]
--------------------------------------------------------------------------------------
(x + 2)^2 * (x + 3)^2 * (x + 1)^4 * (x + 4)^4 * x^6

(5, 1, 18, [2, 2, 4, 4, 6])

[3, 1, 2, 4, -1]

[[3], [1], [2], [4], [-1]]
--------------------------------------------------------------------------------------
(x + 2)^4 * (x + 3)^4 * (x + 1)^6 * (x + 4)^6 * x^11

(10, 1, 31, [4, 4, 6, 6, 11])

[3, 1, 2, 4, -1]

[[3], [1], [2], [4], [-1]]
--------------------------------------------------------------------------------------
(4) * (x + 2)^6 * (x + 3)^6 * (x + 1)^8 * (x + 4)^8 * x^16

(15, 1, 44, [6, 6, 8, 8, 16])

[3, 1, 2, 4, -1]

[[3], [1], [2], [4], [-1]]
--------------------------------------------------------------------------------------
(3) * (x + 2)^8 * (x + 3)^8 * (x + 1)^10 * (x + 4)^10 * x^21

(20, 1, 57, [8, 8, 10, 10, 21])

[3, 1, 2, 4, -1]

[[3], [1], [2], [4], [-1]]
-------------------

In [5]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
residue=1
bound=120
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        zerodata=[]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverFieldTest(poly,prime,sd)
        logs=[]
        digits=[]
        multiplicities=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            multiplicity=lm[j][1]
            digit=log.digits(prime)
            digit= padListOnRightToLengthK(digit,sd)
            logs=logs+[log]
            multiplicities=multiplicities+[multiplicity]
            digits=digits+[digit]
        print
        print (index,sd,td,multiplicities)
        print
        print logs;print;print digits
        print "--------------------------------------------------------------------------------------"

(x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2) * x^2

(1, 4, 6, [1, 1, 1, 1, 2])

[143, 91, 455, 403, -1]

[[3, 3, 0, 1], [1, 3, 3, 0], [0, 1, 3, 3], [3, 0, 1, 3], [-1, 0, 0, 0]]
--------------------------------------------------------------------------------------
(x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2) * (x + 1)^2 * (x + 2)^2 * (x + 3)^2 * (x + 4)^2 * x^7

(6, 4, 19, [1, 1, 1, 1, 2, 2, 2, 2, 7])

[143, 91, 455, 403, 312, 468, 156, 624, -1]

[[3, 3, 0, 1], [1, 3, 3, 0], [0, 1, 3, 3], [3, 0, 1, 3], [2, 2, 2, 2], [3, 3, 3, 3], [1, 1, 1, 1], [4, 4, 4, 4], [-1, 0, 0, 0]]
--------------------------------------------------------------------------------------
(2) * (x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2) * (x + 1)^4 * (x + 2)^4 * (x + 3)^4 * (x + 4)^4 * x^12

(11, 4, 32, [1, 1, 1, 1, 4, 4, 4, 4, 12])

[143, 91

(2) * (x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2) * (x + 1)^38 * (x + 2)^38 * (x + 3)^38 * (x + 4)^38 * x^97

(96, 4, 253, [1, 1, 1, 1, 38, 38, 38, 38, 97])

[143, 91, 455, 403, 312, 468, 156, 624, -1]

[[3, 3, 0, 1], [1, 3, 3, 0], [0, 1, 3, 3], [3, 0, 1, 3], [2, 2, 2, 2], [3, 3, 3, 3], [1, 1, 1, 1], [4, 4, 4, 4], [-1, 0, 0, 0]]
--------------------------------------------------------------------------------------
(3) * (x + 2*t^3 + 4*t + 1) * (x + 2*t^3 + t^2 + t + 3) * (x + 3*t^3 + t + 4) * (x + 3*t^3 + 4*t^2 + 4*t + 2) * (x + 1)^40 * (x + 2)^40 * (x + 3)^40 * (x + 4)^40 * x^102

(101, 4, 266, [1, 1, 1, 1, 40, 40, 40, 40, 102])

[143, 91, 455, 403, 312, 468, 156, 624, -1]

[[3, 3, 0, 1], [1, 3, 3, 0], [0, 1, 3, 3], [3, 0, 1, 3], [2, 2, 2, 2], [3, 3, 3, 3], [1, 1, 1, 1], [4, 4, 4, 4], [-1, 0, 0, 0]]
--------------------------------------------------------------------------------------
(4) * (x + 2*t^3 + 4*t + 1) * (x + 2*t^3 

In [6]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
residue=2
bound=120
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        zerodata=[]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverFieldTest(poly,prime,sd)
        logs=[]
        digits=[]
        multiplicities=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            multiplicity=lm[j][1]
            digit=log.digits(prime)
            digit= padListOnRightToLengthK(digit,sd)
            logs=logs+[log]
            multiplicities=multiplicities+[multiplicity]
            digits=digits+[digit]
        print
        print (index,sd,td,multiplicities)
        print
        print logs;print;print digits
        print "--------------------------------------------------------------------------------------"

(x + 2) * (x + 3) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4) * x^3

(2, 4, 9, [1, 1, 1, 1, 1, 1, 3])

[468, 156, 169, 481, 221, 533, -1]

[[3, 3, 3, 3], [1, 1, 1, 1], [4, 3, 1, 1], [1, 1, 4, 3], [1, 4, 3, 1], [3, 1, 1, 4], [-1, 0, 0, 0]]
--------------------------------------------------------------------------------------
(2) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4) * (x + 1)^2 * (x + 4)^2 * (x + 2)^3 * (x + 3)^3 * x^8

(7, 4, 22, [1, 1, 1, 1, 2, 2, 3, 3, 8])

[169, 481, 221, 533, 312, 624, 468, 156, -1]

[[4, 3, 1, 1], [1, 1, 4, 3], [1, 4, 3, 1], [3, 1, 1, 4], [2, 2, 2, 2], [4, 4, 4, 4], [3, 3, 3, 3], [1, 1, 1, 1], [-1, 0, 0, 0]]
--------------------------------------------------------------------------------------
(2) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4) * (x + 1)^4 * (x + 4)^4 * (x + 2)^5 * (x + 3)^5 * x^13

(12, 4, 35, [1,

(2) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4) * (x + 1)^36 * (x + 4)^36 * (x + 2)^37 * (x + 3)^37 * x^93

(92, 4, 243, [1, 1, 1, 1, 36, 36, 37, 37, 93])

[169, 481, 221, 533, 312, 624, 468, 156, -1]

[[4, 3, 1, 1], [1, 1, 4, 3], [1, 4, 3, 1], [3, 1, 1, 4], [2, 2, 2, 2], [4, 4, 4, 4], [3, 3, 3, 3], [1, 1, 1, 1], [-1, 0, 0, 0]]
--------------------------------------------------------------------------------------
(3) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4) * (x + 1)^38 * (x + 4)^38 * (x + 2)^39 * (x + 3)^39 * x^98

(97, 4, 256, [1, 1, 1, 1, 38, 38, 39, 39, 98])

[169, 481, 221, 533, 312, 624, 468, 156, -1]

[[4, 3, 1, 1], [1, 1, 4, 3], [1, 4, 3, 1], [3, 1, 1, 4], [2, 2, 2, 2], [4, 4, 4, 4], [3, 3, 3, 3], [1, 1, 1, 1], [-1, 0, 0, 0]]
--------------------------------------------------------------------------------------
(3) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t

In [7]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
residue=3
bound=120
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        zerodata=[]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverFieldTest(poly,prime,sd)
        logs=[]
        digits=[]
        multiplicities=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            multiplicity=lm[j][1]
            digit=log.digits(prime)
            digit= padListOnRightToLengthK(digit,sd)
            logs=logs+[log]
            multiplicities=multiplicities+[multiplicity]
            digits=digits+[digit]
        print
        print (index,sd,td,multiplicities)
        print
        print logs;print;print digits
        print "--------------------------------------------------------------------------------------"

(2) * (x + 2) * (x + 3) * (x + 2*t^3 + 3*t^2 + 2*t) * (x + 3*t^3 + 2*t^2 + 3*t) * (x + 2*t^5 + 3*t^3 + 3*t + 1) * (x + 2*t^5 + 2*t^4 + t^2 + t + 1) * (x + 3*t^5 + 2*t^3 + 2*t + 4) * (x + 3*t^5 + 3*t^4 + 4*t^2 + 4*t + 4) * x^4

(3, 6, 12, [1, 1, 1, 1, 1, 1, 1, 1, 4])

[11718, 3906, 10773, 2961, 3717, 14805, 11529, 6993, -1]

[[3, 3, 3, 3, 3, 3], [1, 1, 1, 1, 1, 1], [3, 4, 0, 1, 2, 3], [1, 2, 3, 3, 4, 0], [2, 3, 3, 4, 0, 1], [0, 1, 2, 3, 3, 4], [4, 0, 1, 2, 3, 3], [3, 3, 4, 0, 1, 2], [-1, 0, 0, 0, 0, 0]]
--------------------------------------------------------------------------------------
(4) * (x + 2*t^3 + 3*t^2 + 2*t) * (x + 3*t^3 + 2*t^2 + 3*t) * (x + 2*t^5 + 3*t^3 + 3*t + 1) * (x + 2*t^5 + 2*t^4 + t^2 + t + 1) * (x + 3*t^5 + 2*t^3 + 2*t + 4) * (x + 3*t^5 + 3*t^4 + 4*t^2 + 4*t + 4) * (x + 1)^2 * (x + 4)^2 * (x + 2)^3 * (x + 3)^3 * x^9

(8, 6, 25, [1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 9])

[10773, 2961, 3717, 14805, 11529, 6993, 7812, 15624, 11718, 3906, -1]

[[3, 4, 0, 1, 2, 3], [1, 2, 3, 3


(63, 6, 168, [1, 1, 1, 1, 1, 1, 24, 24, 25, 25, 64])

[10773, 2961, 3717, 14805, 11529, 6993, 7812, 15624, 11718, 3906, -1]

[[3, 4, 0, 1, 2, 3], [1, 2, 3, 3, 4, 0], [2, 3, 3, 4, 0, 1], [0, 1, 2, 3, 3, 4], [4, 0, 1, 2, 3, 3], [3, 3, 4, 0, 1, 2], [2, 2, 2, 2, 2, 2], [4, 4, 4, 4, 4, 4], [3, 3, 3, 3, 3, 3], [1, 1, 1, 1, 1, 1], [-1, 0, 0, 0, 0, 0]]
--------------------------------------------------------------------------------------
(4) * (x + 2*t^3 + 3*t^2 + 2*t) * (x + 3*t^3 + 2*t^2 + 3*t) * (x + 2*t^5 + 3*t^3 + 3*t + 1) * (x + 2*t^5 + 2*t^4 + t^2 + t + 1) * (x + 3*t^5 + 2*t^3 + 2*t + 4) * (x + 3*t^5 + 3*t^4 + 4*t^2 + 4*t + 4) * (x + 1)^26 * (x + 4)^26 * (x + 2)^27 * (x + 3)^27 * x^69

(68, 6, 181, [1, 1, 1, 1, 1, 1, 26, 26, 27, 27, 69])

[10773, 2961, 3717, 14805, 11529, 6993, 7812, 15624, 11718, 3906, -1]

[[3, 4, 0, 1, 2, 3], [1, 2, 3, 3, 4, 0], [2, 3, 3, 4, 0, 1], [0, 1, 2, 3, 3, 4], [4, 0, 1, 2, 3, 3], [3, 3, 4, 0, 1, 2], [2, 2, 2, 2, 2, 2], [4, 4, 4, 4, 4, 4], [3, 3, 3, 3, 3, 3],

In [8]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
residue=4
bound=120
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        zerodata=[]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverFieldTest(poly,prime,sd)
        logs=[]
        digits=[]
        multiplicities=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            multiplicity=lm[j][1]
            digit=log.digits(prime)
            digit= padListOnRightToLengthK(digit,sd)
            logs=logs+[log]
            multiplicities=multiplicities+[multiplicity]
            digits=digits+[digit]
        print
        print (index,sd,td,multiplicities)
        print
        print logs;print;print digits
        print "--------------------------------------------------------------------------------------"

1

(-1, 1, 0, [])

[]

[]
--------------------------------------------------------------------------------------
(4) * (x + 1)^2 * (x + 2)^2 * (x + 3)^2 * (x + 4)^2 * x^5

(4, 1, 13, [2, 2, 2, 2, 5])

[2, 3, 1, 4, -1]

[[2], [3], [1], [4], [-1]]
--------------------------------------------------------------------------------------
(x + 1)^4 * (x + 2)^4 * (x + 3)^4 * (x + 4)^4 * x^10

(9, 1, 26, [4, 4, 4, 4, 10])

[2, 3, 1, 4, -1]

[[2], [3], [1], [4], [-1]]
--------------------------------------------------------------------------------------
(3) * (x + 1)^6 * (x + 2)^6 * (x + 3)^6 * (x + 4)^6 * x^15

(14, 1, 39, [6, 6, 6, 6, 15])

[2, 3, 1, 4, -1]

[[2], [3], [1], [4], [-1]]
--------------------------------------------------------------------------------------
(4) * (x + 1)^8 * (x + 2)^8 * (x + 3)^8 * (x + 4)^8 * x^20

(19, 1, 52, [8, 8, 8, 8, 20])

[2, 3, 1, 4, -1]

[[2], [3], [1], [4], [-1]]
--------------------------------------------------------------------------------------
(3) *